# Prueba 2

`Benjamín Meneses`

## Dataset

- Para esta prueba se utilizará el mismo dataset en todos los ejercicios.
- Cada registro del dataset corresponde a una transacción realizada con una tarjeta Bip!.
- Las columnas son como se describe a continuación:
    - <code>fechahoratrx</code>: Fecha y hora a la en la que se realiza la transacción.
    - <code>codigoentidad</code>: Código del operador.
    - <code>nombreentidad</code>: Nombre del operador.
    - <code>codigositio</code>: Código del lugar en el que se realiza la transacción.
    - <code>nombresitio</code>: Nombre del lugar en el que se realiza la transacción.
    - <code>nrotarjeta</code>: Hash de la tarjeta Bip!
- La ubicación del dataset es <code>s3://bigdata-desafio/transantiago/</code>.
- Los datos se encuentran en formato columnar Parquet

In [1]:
# Iniciamos Spark Application
sc

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
12,application_1643468364668_0013,pyspark3,idle,Link,Link,✔


SparkSession available as 'spark'.
<SparkContext master=yarn appName=livy-session-12>

In [2]:
#Importamos las librerías y comandos necesarios

from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import when

### Ejercicio 1: Identificando usuarios molestosos (4.6 Puntos)
Utilizando el archivo user.json.
Desde Yelp están interesados en identificar a aquellos usuarios que se pueden considerar
como molestosos. Para ello, tienen la siguiente definición de un usuario molestoso:
- Un usuario molestoso es aquél que su promedio de evaluaciones es menor o igual a
2, tiene en promedio menos de 100 reviews y tiene cero fans.

A partir de esta definición, se le solicita los siguientes puntos:
- Identifique en una variable dummy todos los usuarios que se puedan clasificar como
molestosos acorde al criterio.
- Recodificaciones en el archivo user.json:
    - friends, que corresponde a un string con todos los user_id de otros
        usuarios j que siguen al usuario i. El objetivo es contar la cantidad de
        amigos existentes.
    - elite, que corresponde a un string con todos los años en los que el usuario
        i fue considerado como un reviewer de elite. El objetivo es contar la cantidad
        de años en los cuales se consideró como elite.
    - Asegúrese de eliminar los siguientes registros: friends, yelping_since,
        name, elite, user_id.
        
### Requerimientos
Todos los objetivos se deben resolver utilizando pyspark.
- Genere la medición de usuarios molestos en base a los criterios expuestos. (0.8
Puntos).
- Divida la muestra en conjuntos de entrenamiento (preservando un 70% de los datos)
y validación (preservando un 30% de los datos). (0.4 Puntos)
- Entrene tres modelos (LogisticRegression, GBTClassifier y
DecisionTreeClassifier) sin modificar hiperparámetros que en base a los
atributos disponibles en el archivo user.json, clasifique los usuarios molestosos.
(2.2 Puntos)
- Reporte cuál es el mejor modelo en base a la métrica AUC. (0.4 Puntos)
- Identifique cuales son los principales atributos asociados a un usuario molestoso y
repórtelos. (0.8 Puntos)


In [1]:
# Ingresamos todos los .parque y anidamos en 1 solo pyspark DataFrame
df = spark.read.json('s3://bigdata-desafio/yelp-data/user.json')

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
13,application_1643468364668_0014,pyspark3,idle,Link,Link,✔


SparkSession available as 'spark'.


In [4]:
type(df)

<class 'pyspark.sql.dataframe.DataFrame'>

In [5]:
# Mostramos las columnas para familiarizarnos con el DataFrame
df.columns

['average_stars', 'compliment_cool', 'compliment_cute', 'compliment_funny', 'compliment_hot', 'compliment_list', 'compliment_more', 'compliment_note', 'compliment_photos', 'compliment_plain', 'compliment_profile', 'compliment_writer', 'cool', 'elite', 'fans', 'friends', 'funny', 'name', 'review_count', 'useful', 'user_id', 'yelping_since']

In [6]:
# Hacemos un head del dataframe para ver cómo vienen los datos
df.head(1)

[Row(average_stars=4.03, compliment_cool=1, compliment_cute=0, compliment_funny=1, compliment_hot=2, compliment_list=0, compliment_more=0, compliment_note=1, compliment_photos=0, compliment_plain=1, compliment_profile=0, compliment_writer=2, cool=25, elite='2015,2016,2017', fans=5, friends='c78V-rj8NQcQjOI8KP3UEA, alRMgPcngYSCJ5naFRBz5g, ajcnq75Z5xxkvUSmmJ1bCg, BSMAmp2-wMzCkhTfq9ToNg, jka10dk9ygX76hJG0gfPZQ, dut0e4xvme7QSlesOycHQA, l4l5lBnK356zBua7B-UJ6Q, 0HicMOOs-M_gl2eO-zES4Q, _uI57wL2fLyftrcSFpfSGQ, T4_Qd0YWbC3co6WSMw4vxg, iBRoLWPtWmsI1kdbE9ORSA, xjrUcid6Ymq0DoTJELkYyw, GqadWVzJ6At-vgLzK_SKgA, DvB13VJBmSnbFXBVBsKmDA, vRP9nQkYTeNioDjtxZlVhg, gT0A1iN3eeQ8EMAjJhwQtw, 6yCWjFPtp_AD4x93WAwmnw, 1dKzpNnib-JlViKv8_Gt5g, 3Bv4_JxHXq-gVLOxYMQX0Q, ikQyfu1iViYh8T0us7wiFQ, f1GGltNaB7K5DR1jf3dOmg, tgeFUChlh7v8bZFVl2-hjQ, -9-9oyXlqsMG2he5xIWdLQ, Adj9fBPVJad8vSs-mIP7gw, Ce49RY8CKXVsTifxRYFTsw, M1_7TLi8CbdA89nFLlH4iw, wFsNv-hqbW_F5-IRqfBN6g, 0Q1L7zXHocaUZ2gsG2XJeg, cBFgmOCBdhYa0xoFEAzp_g, VrD_AgiFvzqt

In [7]:
# Mostrarmos el tipo de cada dato de cada columna para ver si es necesario cambiar la tipología de alguna
df.printSchema()

root
 |-- average_stars: double (nullable = true)
 |-- compliment_cool: long (nullable = true)
 |-- compliment_cute: long (nullable = true)
 |-- compliment_funny: long (nullable = true)
 |-- compliment_hot: long (nullable = true)
 |-- compliment_list: long (nullable = true)
 |-- compliment_more: long (nullable = true)
 |-- compliment_note: long (nullable = true)
 |-- compliment_photos: long (nullable = true)
 |-- compliment_plain: long (nullable = true)
 |-- compliment_profile: long (nullable = true)
 |-- compliment_writer: long (nullable = true)
 |-- cool: long (nullable = true)
 |-- elite: string (nullable = true)
 |-- fans: long (nullable = true)
 |-- friends: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- name: string (nullable = true)
 |-- review_count: long (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)
 |-- yelping_since: string (nullable = true)

In [8]:
# Gereramos la variable objetivo (molestoso)
df = df.withColumn('molestoso',
                   when((df['fans'] == 0) & (df['average_stars'] <= 2) & (df['review_count'] < 100),
                   1).otherwise(0))

In [9]:
# Cambiamos el df para que tenga la variable Elite y Friends de la forma
# en la que lo necesitamos (contando cada uno de los casos con len(*) y separamos los casos por ","),
# luego lo pasamos nuevamente a un DF

df = df.rdd.map(lambda row: (row['average_stars'],
    row['compliment_cool'],
row['compliment_cute'],
row['compliment_funny'],
row['compliment_hot'],
row['compliment_list'],
row['compliment_more'],
row['compliment_note'],
row['compliment_photos'],
row['compliment_plain'],
row['compliment_profile'],
row['compliment_writer'],
row['cool'],
len(row['elite'].split(',')),
row['fans'],
len(row['friends'].split(',')),
row['funny'],
row['review_count'],
row['useful'],
row['molestoso']                             
)).toDF(
    ['average_stars','compliment_cool',
    'compliment_cute',
    'compliment_funny',
    'compliment_hot',
    'compliment_list',
    'compliment_more',
    'compliment_note',
    'compliment_photos',
    'compliment_plain',
    'compliment_profile',
    'compliment_writer',
    'cool',
    'elite',
    'fans',
    'friends',
    'funny',
    'review_count',
    'useful',
    'molestoso'
    ]
)

In [10]:
# Medimos la cantidad según usuarios molestoso o no molestoso
df.groupBy('molestoso').count().show()

+---------+-------+
|molestoso|  count|
+---------+-------+
|        0|1453462|
|        1| 183676|
+---------+-------+

In [11]:
# Importamos VectorAssembler para poder con él hacer un randomsplit que nos permita separar el dataset
# en parte de train y en parte test
from pyspark.ml.feature import VectorAssembler

# Ahora toca importa las librerías necesarias para generar los algoritmos de clasificación
from pyspark.ml.classification import LogisticRegression, GBTClassifier, DecisionTreeClassifier

In [12]:
# Renombramos el vector objetivo a "label" y creamos la lista "feats" para enlistar las variables que se usarán para predecir
df = df.withColumnRenamed('molestoso', 'label')
feats = df.columns
feats.remove('label')

In [13]:
# Craemos nuestro objeto vec y le indicamos las columnas input, así como también nombramos "assembled_features" como el
# el listado de columnas que usaremos para predecir (Esto es necesario para usar las librerías de los algoritmos)
vec = VectorAssembler(inputCols=feats, outputCol='assembled_features')

In [14]:
# Transformamos nuestro objeto "vec" con la data
vec = vec.transform(df)

In [15]:
# Seleccionamos y nos damos cuenta de que tenemos separado label como el assembled_features
vec = vec.select('label', 'assembled_features')
vec.take(1)

[Row(label=0, assembled_features=DenseVector([4.03, 1.0, 0.0, 1.0, 2.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 2.0, 25.0, 3.0, 5.0, 99.0, 17.0, 95.0, 84.0]))]

In [16]:
# Creamos train y test para entrenar y validar nuestros modelos.
train, test = vec.randomSplit([0.7, 0.3], seed=42)

In [17]:
# Generamos Modelo LogisticRegression
logistic_example = LogisticRegression(featuresCol='assembled_features',
    labelCol='label',
    predictionCol='molestoso_pred')
logistic_example = logistic_example.fit(train)

In [18]:
# Generamos Modelo de GradientBoostingClassifier
gradient_model = GBTClassifier(featuresCol='assembled_features',
    labelCol='label',
    predictionCol='molestoso_pred').fit(train)

In [19]:
# Generamos Modelo DecisionTreeClassifier
decision_tree_model = DecisionTreeClassifier(featuresCol='assembled_features',
    labelCol='label',
    predictionCol='molestoso_pred').fit(train)

In [20]:
#Importamos la librería necesaria para indicarle a cada modelo que genere la columna molestoso_pred
from pyspark.ml.evaluation import BinaryClassificationEvaluator


# Generamos nuestro objeto "evaluator" para que si queremos predecir con un modelo, este genere una 
# columna llamada "molestoso_pred"
evaluator = BinaryClassificationEvaluator()
evaluator.setRawPredictionCol("molestoso_pred")

BinaryClassificationEvaluator_8789fb5725dd

In [21]:
#Dado que todos los modelos el

#generamos un for con los 3 modelos para que nos de el indicar AUC para cada set de validación asociado a cada modelo
models = {
    'Logistic': logistic_example,
    'Gradient': gradient_model,
    'DecisionTree': decision_tree_model
}
for name, model in models.items():
    print('AUC para {}: {}'.format(name, evaluator.evaluate(model.transform(test))))

AUC para Logistic: 0.9999737918990322
AUC para Gradient: 0.9999312599816236
AUC para DecisionTree: 0.9912848237343601

Nos damos cuenta de que el modelo que mejor performa en este ejercicio es Logistic Regression.

In [22]:
# Generamos un for que vaya imprimiendo los coeficientes de la Regresión Logística y nos indique qué variables son las más
# importantes para que el modelo prediga si es molestoso o no.
for name, logodd in zip(feats, list(logistic_example.coefficients)):
    print(name, logodd)

average_stars -155.78639595108254
compliment_cool 0.168527253906581
compliment_cute 0.9102058654597347
compliment_funny 0.168527253906581
compliment_hot 0.17076428850508843
compliment_list 0.8382656982122865
compliment_more 0.04100669477869027
compliment_note -0.2676734606037918
compliment_photos 0.08822867606929702
compliment_plain 0.020375615093030933
compliment_profile -0.5180406407873293
compliment_writer -0.7919166428908563
cool 0.0674008921699163
elite -1829.8814626393062
fans -185.83223987282145
friends -0.0014663230321997173
funny 0.046281684448992194
review_count -0.16222552866331286
useful -0.017149295761152653

Los 4 atributos que mas influyen para el modelo son:

- elite
- fans
- average_stars
- compliment_cute

### Ejercicio 2: Identificando la probabilidad de cierre de un servicio (5.4 Puntos)
Utilizando el archivo business.json.
Desde Yelp están interesados en predecir la probabilidad de cierre de un servicio en base a
los reviews y características de un negocio. Así, la primera iteración del modelo es generar
una identificación de qué factores están asociados al cierre.
El equipo de desarrollo de Yelp le hace entrega de un archivo llamado
recoding_business_schema.py que describe:
- Atributos a recodificar.
- Atributos a mantener.
Este archivo sirve como guía y no implementa la recodificación en el
pyspark.sql.dataframe.DataFrame, esto es tarea de usted.
De manera adicional, cabe destacar que este archivo no incluye la recodificación del vector
objetivo (is_open). Usted deberá recodificarla de manera tal de identificar como 1 aquellos
servicios que cerraron y 0 el resto.

#### Requerimientos
Todos los objetivos se deben resolver utilizando pyspark.
- Implemente el esquema de recodificación. (0.8 Puntos)
- Genere la recodificación del vector objetivo. (0.8 Puntos)
- Divida la muestra en conjuntos de entrenamiento (Preservando un 70% de los datos)
y validación (preservando un 30% de los datos). (0.4 Puntos)
- Entrene tres modelos (LogisticRegression, GBTClassifier y
DecisionTreeClassifier) sin modificar hiperparámetros que en base a los
atributos recodificados del archivo business.json, clasifique aquellos servicios
cerrados. (2.2 Puntos)
- Reporte cuál es el mejor modelo en base a la métrica AUC. (0.4 Puntos)
- Identifique cuales son los principales atributos asociados al cierre de un servicio.
(0.8 Puntos)


In [23]:
# Importamos los datos
df_yelp_business = spark.read.json('s3://bigdata-desafio/yelp-data/business.json')

In [24]:
# Vemos la variable objetivo a recodificar
df_yelp_business.groupBy('is_open').count().show()

+-------+------+
|is_open| count|
+-------+------+
|      0| 34084|
|      1|158525|
+-------+------+

In [25]:
from pyspark.sql.functions import col
# Copiamos la recodificación del archivo recording_business_schema.py en un hash y las iteramos
refactoring = {
    'insurance' : when((col('attributes.AcceptsInsurance') == 'True')\
        | (col('attributes.AcceptsInsurance') == "\'True\'")\
        | (col('attributes.AcceptsInsurance') == "u\'True\'"), 1)\
         .otherwise(0),
    'all_ages_allowed' : when((col('attributes.AgesAllowed') == 'allages')\
        | (col('attributes.AgesAllowed') == "\'allages\'")\
        | (col('attributes.AgesAllowed') == "u\'allages\'"), 1)\
         .otherwise(0),
    'alcohol_consumption' : when((col('attributes.Alcohol') == 'beer_and_wine')\
        | (col('attributes.Alcohol') == "\'beer_and_wine\'")\
        | (col('attributes.Alcohol') == "u\'beer_and_wine\'")\
        | (col('attributes.Alcohol') == 'full_bar')\
        | (col('attributes.Alcohol') == "\'full_bar\'")\
        | (col('attributes.Alcohol') == "u\'full_bar\'"), 1)\
         .otherwise(0),
    'bitcoin_friendly' : when((col('attributes.BusinessAcceptsBitcoin') == 'True')\
        | (col('attributes.BusinessAcceptsBitcoin') == True)\
        | (col('attributes.BusinessAcceptsBitcoin') == "\'True\'")\
        | (col('attributes.BusinessAcceptsBitcoin') == "u\'True\'"), 1)\
         .otherwise(0),
    'food_business' : when((col('categories').rlike('Food'))\
        | (col('categories').rlike('Restaurants'))\
        | (col('categories').rlike('Bars')), 1)\
         .otherwise(0),
    'finance_business' : when((col('categories').rlike('Banks'))\
        | (col('categories').rlike('Insurance'))\
        | (col('categories').rlike('Finance')), 1)\
         .otherwise(0),
    'health_business' : when((col('categories').rlike('Fitness'))\
        | (col('categories').rlike('Hospitals'))\
        | (col('categories').rlike('Health')), 1)\
         .otherwise(0),
    'smokers' : when((col('attributes.Smoking') == '\'yes\'')\
        | (col('attributes.Smoking') == 'u\'yes\'')\
        | (col('attributes.Smoking') == 'yes')\
        | (col('attributes.Smoking') == '\'outdoor\'')\
        | (col('attributes.Smoking') == 'u\'outdoor\'')\
        | (col('attributes.Smoking') == 'outdoor'), 1)\
         .otherwise(0),
    'free_wifi' : when((col('attributes.WiFi') == '\'free\'')\
        | (col('attributes.WiFi') == 'u\'free\'')\
        | (col('attributes.WiFi') == 'free'), 1)\
         .otherwise(0),
    'splurge' : when((col('attributes.RestaurantsPriceRange2') == 3)\
        | (col('attributes.RestaurantsPriceRange2') == 4), 1)\
         .otherwise(0),
    'kids_friendly' : when((col('attributes.GoodForKids') == 'True')\
        | (col('attributes.GoodForKids') == True)\
        | (col('attributes.GoodForKids') == "\'True\'")\
        | (col('attributes.GoodForKids') == "u\'True\'"), 1)\
         .otherwise(0),
    'has_tv' : when((col('attributes.HasTV') == 'True')\
        | (col('attributes.HasTV') == True)\
        | (col('attributes.HasTV') == "\'True\'")\
        | (col('attributes.HasTV') == "u\'True\'"), 1)\
         .otherwise(0),
    'dogs_friendly' : when((col('attributes.DogsAllowed') == 'True')\
        | (col('attributes.DogsAllowed') == True)\
        | (col('attributes.DogsAllowed') == "\True'\'")\
        | (col('attributes.DogsAllowed') == "u\'True\'"), 1)\
         .otherwise(0),
    'loud_place' : when((col('attributes.NoiseLevel') == 'loud')\
        | (col('attributes.NoiseLevel') == "\'loud\'")\
        | (col('attributes.NoiseLevel') == "u\'loud\'")\
        | (col('attributes.NoiseLevel') == "very_loud")\
        | (col('attributes.NoiseLevel') == "\'very_loud\'")\
        | (col('attributes.NoiseLevel') == "u\'very_loud\'"), 1)\
         .otherwise(0),
    'happy_hour' : when((col('attributes.HappyHour') == 'True')\
        | (col('attributes.HappyHour') == True)\
        | (col('attributes.HappyHour') == "\'True\'")\
        | (col('attributes.HappyHour') == "u\'True\'"), 1)\
         .otherwise(0),
    'is_open_recoded' : when((col('is_open') == 0), 1)\
         .otherwise(0)
}
for colName, refactor in refactoring.items():
    df_yelp_business = df_yelp_business.withColumn(colName, refactor)

In [26]:
df_yelp_business.take(2)

[Row(address='2818 E Camino Acequia Drive', attributes=Row(AcceptsInsurance=None, AgesAllowed=None, Alcohol=None, Ambience=None, BYOB=None, BYOBCorkage=None, BestNights=None, BikeParking=None, BusinessAcceptsBitcoin=None, BusinessAcceptsCreditCards=None, BusinessParking=None, ByAppointmentOnly=None, Caters=None, CoatCheck=None, Corkage=None, DietaryRestrictions=None, DogsAllowed=None, DriveThru=None, GoodForDancing=None, GoodForKids='False', GoodForMeal=None, HairSpecializesIn=None, HappyHour=None, HasTV=None, Music=None, NoiseLevel=None, Open24Hours=None, OutdoorSeating=None, RestaurantsAttire=None, RestaurantsCounterService=None, RestaurantsDelivery=None, RestaurantsGoodForGroups=None, RestaurantsPriceRange2=None, RestaurantsReservations=None, RestaurantsTableService=None, RestaurantsTakeOut=None, Smoking=None, WheelchairAccessible=None, WiFi=None), business_id='1SWheh84yJXfytovILXOAQ', categories='Golf, Active Life', city='Phoenix', hours=None, is_open=0, latitude=33.5221425, longit

In [39]:
# Refactorizamos quedándonos sólo con las columnas necesarias
df_yelp_business = df_yelp_business.rdd\
    .map(lambda row: (
        row['is_open_recoded'],
        row['review_count'],
        row['stars'],
        row['insurance'],
        row['all_ages_allowed'],
        row['alcohol_consumption'],
        row['bitcoin_friendly'],
        row['food_business'],
        row['finance_business'],
        row['health_business'],
        row['smokers'],
        row['free_wifi'],
        row['splurge'],
        row['kids_friendly'],
        row['has_tv'],
        row['dogs_friendly'],
        row['loud_place'],
        row['happy_hour']))\
    .toDF([
       'label',
       'review_count',
       'stars',
       'insurance',
       'all_ages_allowed',
       'alcohol_consumption',
       'bitcoin_friendly',
       'food_business',
       'finance_business',
       'health_business',
       'smokers',
       'free_wifi',
       'splurge',
       'kids_friendly',
       'has_tv',
       'dogs_friendly',
       'loud_place',
       'happy_hour',                                       
    ])

In [41]:
# Separamos el espacio de atributos del vector objetivo
features = df_yelp_business.columns
features.remove('label')
# Vectorizamos y generamos las muestras de entrenamiento y testing
af = VectorAssembler(inputCols = features, outputCol = 'assembled_features')
af = af.transform(df_yelp_business)
af = af.select('label', 'assembled_features')
train, test = af.randomSplit([0.7, 0.3])

In [42]:
# Instanciamos el evaluador para nuestros modelos
evaluator = BinaryClassificationEvaluator()
evaluator.setRawPredictionCol("is_open_recoded_pred")

BinaryClassificationEvaluator_adbaa6c71fae

In [43]:
# Hacemos El modelo de regresión Logística
model_logreg = LogisticRegression(featuresCol = 'assembled_features', labelCol = 'label', predictionCol = 'is_open_recoded_pred')
model_logreg = model_logreg.fit(train)
print('AUC de modelo {}: {}'.format('Logistic', evaluator.evaluate(model_logreg.transform(test))))

AUC de modelo Logistic: 0.5103141146535245

In [44]:
# Hacemos El modelo de gradient boosted trees
model_gboostclassifier = GBTClassifier(featuresCol = 'assembled_features', labelCol = 'label', predictionCol = 'is_open_recoded_pred')
model_gboostclassifier = model_gboostclassifier.fit(train)
print('AUC de modelo {}: {}'.format('GradientBoostTree', evaluator.evaluate(model_gboostclassifier.transform(test))))

AUC de modelo GradientBoostTree: 0.538707826411734

In [45]:
# Hacemos El modelo de Decision Trees
model_dectreeclassifier = DecisionTreeClassifier(featuresCol = 'assembled_features', labelCol = 'label', predictionCol = 'is_open_recoded_pred')
model_dectreeclassifier = model_dectreeclassifier.fit(train)
print('AUC de modelo {}: {}'.format('DecisionTree', evaluator.evaluate(model_dectreeclassifier.transform(test))))

AUC de modelo DecisionTree: 0.5292487091686493

Vemos que el mejor modelo es el de GradientBoostTree

In [73]:
# Verificamos las importancias de cada atributo para el mejor modelo
importances = list(zip(features, list(model_gboostclassifier.featureImportances)))
importances.sort(reverse=True, key=lambda x: x[1])
for columna, importance in importances:
    print('La importancia de {0} en el modelo es de {1} '.format(columna, importance))

La importancia de review_count en el modelo es de 0.2064726695212929 
La importancia de stars en el modelo es de 0.18008733291171436 
La importancia de food_business en el modelo es de 0.15383179005304842 
La importancia de alcohol_consumption en el modelo es de 0.0916557104657742 
La importancia de kids_friendly en el modelo es de 0.08510811343631108 
La importancia de insurance en el modelo es de 0.07483922751961308 
La importancia de happy_hour en el modelo es de 0.03836560982941421 
La importancia de splurge en el modelo es de 0.03021781803063885 
La importancia de has_tv en el modelo es de 0.028669996422767796 
La importancia de finance_business en el modelo es de 0.02555553131716045 
La importancia de free_wifi en el modelo es de 0.02072203694780641 
La importancia de health_business en el modelo es de 0.017481260214469184 
La importancia de loud_place en el modelo es de 0.015924394479092514 
La importancia de dogs_friendly en el modelo es de 0.014860260177638678 
La importancia 